In [1]:
import csv

lines = []
with open("./data/driving_log.csv") as file:
    reader = csv.reader(file)
    for line in reader:
        lines.append(line)
        
#ignore the first row with column headings --> center,left,right,steering,throttle,brake,speed
lines = lines[1:]

In [2]:
import cv2
import numpy as np

images = []
steer = []
for _ in lines:
    image = cv2.imread("./data/" + _[0])
    images.append(image)
    ste = float(_[3])
    steer.append(ste)
    
x_train = np.array(images)
y_train = np.array(steer)

In [ ]:
from keras.models import Sequential
from keras.layers import Flatten , Dense , Lambda , Convolution2D , Dropout , Activation
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.layers.pooling import MaxPooling2D


model = Sequential()
model.add(Lambda(lambda x: x/255.0 -0.5 , input_shape=(160 , 320 ,3 )))
# model.add(Flatten())

model.add(Convolution2D(32, 3, 3, input_shape=(160, 320, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))

model.add(Dense(1))

model.compile(loss = 'mse' , optimizer='adam')
# model.fit(x_train , y_train , validation_split=0.1 , shuffle=True , nb_epoch=12)
# model.save('model.h5')

filepath="model.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early_stop = EarlyStopping(monitor='val_loss', patience=3, mode='min') 
callbacks_list = [checkpoint, early_stop]
model.fit(x_train, y_train,  validation_split=0.1 , nb_epoch=100, callbacks=callbacks_list ,batch_size=256)

Train on 7232 samples, validate on 804 samples
Epoch 1/100
 256/7232 [>.............................] - ETA: 5813s - loss: 0.0390